In [6]:
from dotenv import load_dotenv
import os
import requests
import json
from openai import OpenAI

load_dotenv()

class CompanyInfoExtractor:
    def __init__(self):
        self.tavily_api_key = os.getenv("TAVILY_API_KEY")
        self.openai_api_key = os.getenv("OPENAI_API_KEY")

        if not self.tavily_api_key:
            raise ValueError("❌ TAVILY_API_KEY 환경변수가 설정되지 않았습니다.")
        if not self.openai_api_key:
            raise ValueError("❌ OPENAI_API_KEY 환경변수가 설정되지 않았습니다.")

        self.openai_client = OpenAI(api_key=self.openai_api_key)

    def generate_queries(self, company_name):
        return {
            "industry": f"{company_name} 산업 분야",
            "sales": f"{company_name} 연간 매출",
            "total_funding": f"{company_name} 총 투자 유치 금액",
            "homepage": f"{company_name} 공식 홈페이지",
            "key_executive": f"{company_name} CEO",
            "address": f"{company_name} 회사 주소",
            "email": f"{company_name} 연락 이메일",
            "phone_number": f"{company_name} 연락처",
            "company_description": f"{company_name} 회사 설명",
            "products_services": f"{company_name} 주요 제품 및 서비스",
            "target_customers": f"{company_name} 주요 타겟 고객층",
            "competitors": f"{company_name} 주요 경쟁사",
            "strengths": f"{company_name} 강점",
            "business_model": f"{company_name} 비즈니스 모델"
        }

    def search_tavily(self, query, num_results=3):
        url = "https://api.tavily.com/search"
        headers = {
            "Authorization": f"Bearer {self.tavily_api_key}",
            "Content-Type": "application/json"
        }
        payload = {
            "query": query,
            "search_depth": "basic"
        }
        try:
            response = requests.post(url, headers=headers, json=payload)
            response.raise_for_status()
            results = response.json().get("results", [])[:num_results]
            
            # ✅ 반드시 이렇게: content와 url을 모두 포함
            return [
                {"url": r.get("url"), "content": r.get("content", "")}
                for r in results if "content" in r and "url" in r
            ]
            
        except requests.HTTPError as http_err:
            print(f"🔐 Tavily 인증 실패 또는 요청 오류: {http_err}")
            return []
        except Exception as e:
            print(f"⚠️ Tavily 검색 실패: {e}")
            return []
        
    def extract_info(self, company_name):
        queries = self.generate_queries(company_name)
        extracted_info = {}

        for field, query in queries.items():
            print(f"🔍 {field} → 검색 쿼리: {query}")
            sources = self.search_tavily(query)
            if not sources:
                print(f"⚠️ {field} 관련된 content 없음 (패스)")
                continue

            contents = [s["content"] for s in sources]
            urls = [s["url"] for s in sources]

            combined_content = "\n\n".join(contents)

            prompt = f"""
    당신은 회사 분석 전문가이며, 당신은 '{company_name}'에 대해 이미 내부 지식을 어느 정도 알고 있습니다.
    하지만 더 정확하고 최신 정보를 제공하기 위해 아래의 웹 검색 결과도 함께 참고해야 합니다.

    당신의 작업 목표는:
    - '{field}' 항목에 대해 정확한 정보를 JSON 형식으로 추출하는 것입니다.
    - 당신의 내부 지식과 아래 제공된 외부 웹 검색 내용을 통합해서 판단하세요.
    - 단, 최신 정보나 수치(예: 매출, 대표자 등)는 아래 텍스트를 우선적으로 신뢰하세요.
    - 정보가 명확하지 않을 경우 null 또는 "정보 없음"으로 처리하세요.
    - 최대한 정확한 정보들을 많이 제공하세요.

    회사명: "{company_name}"
    요청 항목: "{field}"

    # 참고용 외부 텍스트:
    {combined_content}

    # 응답 형식 (JSON만 반환):
    {{
    "{field}": ...
    }}
    """

            try:
                completion = self.openai_client.chat.completions.create(
                    model="gpt-4.1-mini",
                    messages=[
                        {"role": "system", "content": "You are a structured data extractor."},
                        {"role": "user", "content": prompt}
                    ],
                    temperature=0.3,
                    response_format={"type": "json_object"}
                )
                field_result = json.loads(completion.choices[0].message.content)
                extracted_info[field] = field_result
                extracted_info[f"{field}_sources"] = urls # URL 추가
            except Exception as e:
                print(f"❌ LLM 추출 실패 [{field}]: {e}")
                continue

        return extracted_info


if __name__ == "__main__":
    extractor = CompanyInfoExtractor()
    result = extractor.extract_info("토스")
    print(json.dumps(result, indent=2, ensure_ascii=False))


🔍 industry → 검색 쿼리: 토스 산업 분야
🔍 sales → 검색 쿼리: 토스 연간 매출
🔍 total_funding → 검색 쿼리: 토스 총 투자 유치 금액
🔍 homepage → 검색 쿼리: 토스 공식 홈페이지
🔍 key_executive → 검색 쿼리: 토스 CEO
🔍 address → 검색 쿼리: 토스 회사 주소
🔍 email → 검색 쿼리: 토스 연락 이메일
🔍 phone_number → 검색 쿼리: 토스 연락처
🔍 company_description → 검색 쿼리: 토스 회사 설명
🔍 products_services → 검색 쿼리: 토스 주요 제품 및 서비스
🔍 target_customers → 검색 쿼리: 토스 주요 타겟 고객층
🔍 competitors → 검색 쿼리: 토스 주요 경쟁사
🔍 strengths → 검색 쿼리: 토스 강점
🔍 business_model → 검색 쿼리: 토스 비즈니스 모델
{
  "industry": {
    "industry": "금융 종합 핀테크"
  },
  "industry_sources": [
    "https://rich365.co.kr/entry/비바-리퍼블리카토스-기업-정보-비바-리퍼블리카-사업분야-상장전망-실적-정리",
    "https://blog.toss.im/article/head-inside-business",
    "https://byline.network/2022/04/04-63/"
  ],
  "sales": {
    "sales": {
      "2023": {
        "consolidated_revenue": "13,706억원",
        "revenue_breakdown": {
          "consumer_segment": "42.5%",
          "merchant_segment": "57.5%"
        }
      },
      "2024": {
        "consolidated_revenue": "19,556억원",
  